In [2]:
# Psycopg2 is a popular PostgreSQL adapter for the Python programming language. It allows Python code to interact with PostgreSQL databases. 
import psycopg2

# Amazon SageMaker and related imports
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer
from sagemaker import get_execution_role
from sagemaker import image_uris
import boto3  # AWS Python sdk

%matplotlib inline

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
PGEND_POINT = 'database-1.cfwmkaw8o6bp.us-east-1.rds.amazonaws.com' #End Point
PGDATABASE_NAME ='heart_attack_prediction_db' #data base name 
PGUSER_NAME = 'postgres'
PGPASSWORD = 'B00TC4MP'

In [4]:
def connect():
    #conn_string = "host="+ PGEND_POINT +" port="+ "5432" + "bdname=" + PGDATABASE_NAME + " user=" + PGUSER_NAME + " password=" + PGPASSWORD
    conn_string = f"host={PGEND_POINT} port=5432 dbname={PGDATABASE_NAME} user={PGUSER_NAME} password={PGPASSWORD}"
    conn = psycopg2.connect(conn_string)
    print("Connected!")
    
    #Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor


In [5]:
#Close connection function definition
def close_connection(conn, cursor):
    conn.commit()
    cursor.close()
    conn.close()
    print("Connection closed.")

In [6]:
conn, cursor = connect()

Connected!


In [7]:
# Call this function when you're done with your database operations
close_connection(conn, cursor)

Connection closed.


# TEST TABLE INSERT

In [8]:
conn, cursor = connect()

Connected!


In [9]:
from psycopg2 import sql

query_food = sql.SQL("""CREATE TABLE Menu(
    food_id INT NOT NULL,
    food_name VARCHAR(30) NOT NULL, 
    PRIMARY KEY (food_id),
    UNIQUE(food_name)
    );""")

In [10]:
cur = conn.cursor()
cur.execute(query_food)
cur.close() #close the communication with the PostgresSQL database server
conn.commit() #Commit changes

In [11]:
#insert some data into table
query_food_insert = sql.SQL("""
INSERT INTO Menu
    (food_id, food_name) VALUES (1, 'Tacos'), (2,'Tomato Soup'), (3, 'Grilled Cheese'); 
    """)

In [12]:
cur = conn.cursor()
cur.execute(query_food_insert)
cur.close() #close the communication with the PostgresSQL database server
conn.commit() #Commit changes

In [13]:
#query the table
query_menu = sql.SQL("""
SELECT * FROM Menu;""")

In [19]:
cur = conn.cursor()
cur.execute(query_menu)

In [21]:
print(cur.fetchone())

(1, 'Tacos')


In [17]:
print(cur.fetchall())

[(1, 'Tacos'), (2, 'Tomato Soup'), (3, 'Grilled Cheese')]


# DATA FRAME

In [ ]:
cur = conn.cursor()
cur.execute(query_menu)

In [23]:
#importing data set using pandas
import pandas as pd
df_menu = pd.DataFrame(cur.fetchall(), columns=['food_id','food_name'])

In [24]:
df_menu

,food_id,food_name
0,2,Tomato Soup
1,3,Grilled Cheese


In [25]:
cur.close()
conn.commit()

# Data Frame into a Table

In [26]:
data = {'Name':['Tom','Jerry','Spedee'], 'Age':[50, 25, 10]}

In [27]:
df_people = pd.DataFrame(data)
df_people

,Name,Age
0,Tom,50
1,Jerry,25
2,Spedee,10


In [35]:
query_people = sql.SQL("""CREATE TABLE people (
    Name VARCHAR(30) NOT NULL, 
    Age INT NOT NULL, 
    PRIMARY KEY(Name),
    UNIQUE (Name)
);""")

In [36]:
cur = conn.cursor()
cur.execute(query_people)
cur.close() #close the communication with the PostgresSQL database server
conn.commit() #Commit changes

In [42]:
#single insert 
def single_inserts(conn, df, table):
    for i in df.index:
        cur = conn.cursor()
        cols = ','.join(list(df.columns))
        vals = [df.at[i,col] for col in list(df.columns)]
        query = "INSERT INTO %s(%s) VALUES('%s',%s)" %(table, cols, vals[0], vals[1])
        cur.execute(query)
        cur.close()
        conn.commit()
    print("single insert completed")

In [43]:
single_inserts(conn, df_people, 'people')

single insert completed


In [44]:
query_people_table = sql.SQL("""
SELECT * FROM people;
""")

In [45]:
cur = conn.cursor()
cur.execute(query_people_table)
print(cur.fetchall())
cur.close()
conn.commit()

[('Tom', 50), ('Jerry', 25), ('Spedee', 10)]


In [34]:
#WARNING THIS DROP THE TABLE
query_drop = sql.SQL("DROP TABLE IF EXISTS people;")
conn, cursor = connect()
cur = conn.cursor()
cur.execute(query_drop)
cur.close()
conn.commit()

Connected!


# batch insert
 * Importing large data sets into our data base

In [95]:
#importing data set using pandas
import pandas as pd

heart_attack_df = pd.read_csv("Resources/heart_attack_prediction_dataset.csv")

In [96]:
heart_attack_df.shape

(8763, 26)

In [97]:
heart_attack_df.sample(5)

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
2430,SQU8423,42,Female,260,149/65,93,0,1,1,0,...,0.796202,27113,23.235400,322,3,9,Vietnam,Asia,Northern Hemisphere,0
4365,KZJ3624,44,Male,261,98/85,104,1,1,1,1,...,2.345751,276643,39.199193,747,6,5,Colombia,South America,Northern Hemisphere,1
7666,FFQ1643,78,Female,259,172/72,75,0,1,1,1,...,4.159616,187050,27.108105,447,7,7,France,Europe,Northern Hemisphere,1
6582,AUU5753,26,Female,275,134/80,104,0,0,0,1,...,1.478529,274496,23.080689,758,6,6,India,Asia,Northern Hemisphere,0
3798,DJM6628,54,Male,146,163/108,93,0,0,1,1,...,8.139367,67895,35.173643,593,0,9,Brazil,South America,Southern Hemisphere,0


In [98]:
heart_attack_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [99]:
# Changing the name of the columns
heart_attack_df.columns = heart_attack_df.columns.str.replace(' ', '_')

In [100]:
heart_attack_df.sample(5)

,Patient_ID,Age,Sex,Cholesterol,Blood_Pressure,Heart_Rate,Diabetes,Family_History,Smoking,Obesity,...,Sedentary_Hours_Per_Day,Income,BMI,Triglycerides,Physical_Activity_Days_Per_Week,Sleep_Hours_Per_Day,Country,Continent,Hemisphere,Heart_Attack_Risk
7542,VDR4043,63,Female,374,169/96,93,1,1,1,0,...,6.152517,67278,35.999614,204,6,10,United States,North America,Northern Hemisphere,0
4447,BKA0472,70,Male,289,112/94,44,1,1,1,0,...,0.513624,94392,35.886313,566,3,5,Japan,Asia,Northern Hemisphere,0
6170,KRG4242,60,Male,398,103/65,68,0,0,1,0,...,9.583907,281319,38.857619,638,4,7,China,Asia,Northern Hemisphere,1
4872,OYT8686,28,Female,210,94/108,69,1,1,0,0,...,11.585917,143774,29.962606,648,5,10,Australia,Australia,Southern Hemisphere,0
3026,MIB3612,66,Female,305,174/108,106,1,1,1,0,...,7.150994,76551,33.525833,362,3,10,Australia,Australia,Southern Hemisphere,1


In [101]:
# Convert column names to lowercase
heart_attack_df.columns = heart_attack_df.columns.str.lower()
heart_attack_df.sample(5)

,patient_id,age,sex,cholesterol,blood_pressure,heart_rate,diabetes,family_history,smoking,obesity,...,sedentary_hours_per_day,income,bmi,triglycerides,physical_activity_days_per_week,sleep_hours_per_day,country,continent,hemisphere,heart_attack_risk
4682,ZPI7839,34,Female,337,143/99,47,0,1,0,1,...,11.571365,255504,27.044583,42,2,10,United Kingdom,Europe,Northern Hemisphere,0
4598,KGL3058,18,Male,174,106/68,53,0,0,1,0,...,6.998715,207143,31.822149,94,2,10,Argentina,South America,Southern Hemisphere,0
5619,UCF5776,42,Female,312,116/105,60,0,1,1,1,...,11.978335,289517,32.907530,507,4,9,Australia,Australia,Southern Hemisphere,1
5345,KRW0690,41,Male,199,135/66,75,0,0,1,1,...,0.286899,165990,18.909464,174,2,7,Australia,Australia,Southern Hemisphere,0
7911,NTO0029,23,Male,261,98/97,64,1,0,1,1,...,5.085240,105057,27.699493,288,6,10,China,Asia,Northern Hemisphere,1


In [102]:
from psycopg2 import sql

query_hat = sql.SQL("""
    CREATE TABLE HeartAttackPrediction (
        patient_id VARCHAR(255) PRIMARY KEY,
        age INT,
        sex VARCHAR(255),
        cholesterol INT,
        blood_pressure VARCHAR(255),
        heart_rate INT,
        diabetes INT,
        family_history INT,
        smoking INT,
        obesity INT,
        alcohol_consumption INT,
        exercise_hours_per_week FLOAT,
        diet VARCHAR(255), 
        previous_heart_problems INT,  
        medication_use INT,  
        stress_level INT,
        sedentary_hours_per_day FLOAT,
        income INT,
        bmi FLOAT,
        triglycerides INT,
        physical_activity_days_per_week INT,
        sleep_hours_per_day INT,
        country VARCHAR(255),
        continent VARCHAR(255),
        hemisphere VARCHAR(255),
        heart_attack_risk INT
    );""")

In [103]:
conn, cursor = connect()
cur = conn.cursor()
cur.execute(query_hat)
cur.close()
conn.commit()
print("Table created successfully")

Connected!
Table created successfully


In [104]:
def table_exists(table_name):
    query = sql.SQL("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = %s);")
    conn, cursor = connect()
    cursor.execute(query, [table_name])
    exists = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    return exists

In [105]:
# Call this function to review the table
table_name = "heartattackprediction"
if table_exists(table_name):
    print(f"The table {table_name} exists in the database.")
else:
    print(f"The table {table_name} does not exist in the database.")

Connected!
The table heartattackprediction exists in the database.


In [107]:
def update_many(conn, datafrm, table):
    #creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    #dataframe colummns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    #SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, %%s,%%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s,%%s, %%s, %%s, %%s, %%s, %%s, %%s)" % (table, cols)
    #sql = "INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, %%s,'%%s', %%s, %%s, %%s, %%s, %%s, %%s, %%s, '%%s', %%s, %%s, %%s, %%s, %%s, %%s,%%s, %%s, %%s, '%%s', '%%s', '%%s', %%s)" % (table, cols)
    #sql = "INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, %%s,%%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    cursor.executemany(sql, tpls)
    print("Data upload complete successfully...")
    cursor.close()
    conn.commit()

In [108]:
update_many(conn, heart_attack_df, 'heartattackprediction')

Data upload complete successfully...


In [111]:
query_hat_all = sql.SQL("""
SELECT * FROM heartattackprediction;
""")

In [112]:
cur = conn.cursor()
cur.execute(query_hat_all)

In [113]:
print(cur.fetchone())

('BMW7812', 67, 'Male', 208, '158/88', 72, 0, 0, 1, 0, 0, 4.168188835442079, 'Average', 0, 0, 9, 6.61500145291406, 261404, 31.2512327252954, 286, 0, 6, 'Argentina', 'South America', 'Southern Hemisphere', 0)


In [ ]:
#THIS PRINT ALL THE INFORMATION ON THE TABLE 
print(cur.fetchall())
cur.close()
conn.commit()

In [118]:
cur = conn.cursor()
cur.execute(query_hat_all)
hat_df = pd.DataFrame(cur.fetchall(), columns=['Patient ID', 'Age', 'Sex','Cholesterol', 'Blood Pressure','Heart Rate','Diabetes','Family History','Smoking','Obesity','Alcohol Consumption','Exercise Hours Per Week','Diet','Previous Heart Problems','Medication Use','Stress Level','Sedentary Hours Per Day','Income','BMI','Triglycerides','Physical Activity Days Per Week','Sleep Hours Per Day','Country','Continent','Hemisphere','Heart Attack Risk'])

In [119]:
hat_df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [120]:
hat_df.shape

(8763, 26)

In [122]:
hat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [123]:
# Call this function when you're done with your database operations
close_connection(conn, cursor)

Connection closed.


In [94]:
#WARNING THIS SCRIPT DROP THE TABLE HeartAttackPrediction
query_drop = sql.SQL("DROP TABLE IF EXISTS HeartAttackPrediction;")
conn, cursor = connect()
cur = conn.cursor()
cur.execute(query_drop)
cur.close()
conn.commit()

Connected!
